In [10]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
%matplotlib inline



In [11]:
# GoogleFinance files for My Portfolio
aapl_data = Path("Sources/Apple stock.csv")
msft_data = Path("Sources/Microsoft stock.csv")
csco_data = Path("Sources/Cisco stock.csv")
goog_data = Path("Sources/Google stock.csv")
intl_data = Path("Sources/Intel stock.csv")
nvda_data = Path("Sources/Nvidia stock.csv")
orcl_data = Path("Sources/Oracle stock.csv")
tex_data = Path("Sources/Tex Ins stock.csv")
ESG_data = Path("Sources/ESG Scores.csv")
tex_data

PosixPath('Sources/Tex Ins stock.csv')

In [12]:
# Read the individual CSV datasets 
apple = pd.read_csv(aapl_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
msft = pd.read_csv(msft_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
cisco = pd.read_csv(csco_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
google = pd.read_csv(goog_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
intel = pd.read_csv(intl_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
nvidia = pd.read_csv(nvda_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
oracle = pd.read_csv(orcl_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
tex_ins = pd.read_csv(tex_data, index_col="Date",infer_datetime_format=True, parse_dates=True)
ESG = pd.read_csv(ESG_data, index_col="Fiscal Year End Date",infer_datetime_format=True, parse_dates=True)
ESG

,Organization ID,Fiscal Year,Special Value Flag,Feed Date,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,...,Asset4 Identifier,Organization ID.1,Refinitiv Business Classification Code,Primary Exchange,Status Flag (1 = Active),Organization Authority Identifier,Ticker,CUSIP,Sedol,ISIN
Fiscal Year End Date,,,,,,,,,,,,,,,,,,,,,
2010-12-25,18227,2010,500,9/25/2022,0.951624,0.515286,0.078947,0.953333,0.992754,0.833333,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2011-12-31,18227,2011,500,9/25/2022,0.905026,0.640013,0.375000,0.980263,0.979452,0.776923,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2012-12-29,18227,2012,500,9/25/2022,0.913084,0.913084,0.958333,0.981250,0.993333,0.778571,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2013-12-28,18227,2013,500,9/25/2022,0.856952,0.653476,0.450000,0.981250,0.993243,0.787671,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2014-12-27,18227,2014,500,9/25/2022,0.913875,0.648114,0.382353,0.981707,0.993421,0.783784,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-28,119895127,2018,500,9/25/2022,0.882782,0.649724,0.416667,0.989130,0.988636,0.426471,...,A4_00069,Cisco Systems Inc,5710,NSM,1,5080018615,CSCO,17275R10,219816,US17275R1023
2019-07-27,119895127,2019,500,9/25/2022,0.879063,0.879063,0.882353,0.991803,0.973214,0.372093,...,A4_00069,Cisco Systems Inc,5710,NSM,1,5080018615,CSCO,17275R10,219816,US17275R1023
2020-07-25,119895127,2020,500,9/25/2022,0.892640,0.543542,0.194444,0.993421,0.978261,0.392157,...,A4_00069,Cisco Systems Inc,5710,NSM,1,5080018615,CSCO,17275R10,219816,US17275R1023


In [13]:
#Create My portfolio pivot table
stock_performance = pd.concat([apple["Close"], msft["Close"], cisco["Close"], google["Close"], intel["Close"], nvidia["Close"], oracle["Close"], tex_ins["Close"]], axis="columns", join="inner")


# Sort datetime index in ascending order (past to present)
stock_performance.sort_index(inplace=True)

# Set column names 
stock_performance.columns = ["APPLE", "MSFT", "CISCO", "GOOGLE", "INTEL", "NVIDIA", "ORACLE", "TEX INS"]

# Display a few lines
stock_performance

,APPLE,MSFT,CISCO,GOOGLE,INTEL,NVIDIA,ORACLE,TEX INS
Date,,,,,,,,
2014-03-27 16:00:00,19.20,39.36,22.02,27.85,25.31,4.45,39.24,46.08
2014-03-28 16:00:00,19.17,40.30,22.33,27.92,25.62,4.47,39.57,46.64
2014-03-31 16:00:00,19.17,40.99,22.42,27.77,25.81,4.48,40.91,47.16
2014-04-01 16:00:00,19.34,41.42,23.10,28.28,25.99,4.66,41.49,47.53
2014-04-02 16:00:00,19.38,41.35,22.99,28.27,25.89,4.63,41.13,47.62
...,...,...,...,...,...,...,...,...
2022-12-23 16:00:00,131.86,238.73,47.48,89.81,26.09,152.06,80.85,164.38
2022-12-27 16:00:00,130.03,236.96,47.53,87.93,25.94,141.21,80.98,163.78
2022-12-28 16:00:00,126.04,234.53,47.07,86.46,25.54,140.36,80.33,161.19


In [14]:
# Removes duplicates and drops NAs from stock_performance file
stock_performance = stock_performance.dropna()
stock_performance = stock_performance.drop_duplicates()

In [15]:
# Calculates daily returns
stock_performance = stock_performance.pct_change().dropna()
stock_performance

,APPLE,MSFT,CISCO,GOOGLE,INTEL,NVIDIA,ORACLE,TEX INS
Date,,,,,,,,
2014-03-28 16:00:00,-0.001562,0.023882,0.014078,0.002513,0.012248,0.004494,0.008410,0.012153
2014-03-31 16:00:00,0.000000,0.017122,0.004030,-0.005372,0.007416,0.002237,0.033864,0.011149
2014-04-01 16:00:00,0.008868,0.010490,0.030330,0.018365,0.006974,0.040179,0.014177,0.007846
2014-04-02 16:00:00,0.002068,-0.001690,-0.004762,-0.000354,-0.003848,-0.006438,-0.008677,0.001894
2014-04-03 16:00:00,-0.007224,-0.008222,0.004350,0.004952,0.020085,0.010799,-0.018478,0.001470
...,...,...,...,...,...,...,...,...
2022-12-23 16:00:00,-0.002798,0.002267,0.003381,0.017562,0.004621,-0.008671,0.004473,-0.001943
2022-12-27 16:00:00,-0.013878,-0.007414,0.001053,-0.020933,-0.005749,-0.071353,0.001608,-0.003650
2022-12-28 16:00:00,-0.030685,-0.010255,-0.009678,-0.016718,-0.015420,-0.006019,-0.008027,-0.015814


In [16]:
# Inspect nulls and NAs
ESG.isna().sum()

Organization ID                                      0
Fiscal Year                                          0
Special Value Flag                                   0
Feed Date                                            0
ESG SCORE                                            7
ESG Combined Score                                   7
ESG Controversies Score                              7
Resource Use Score                                   7
Emissions Score                                      7
Environmental Innovation Score                       7
Workforce Score                                      7
Human Rights Score                                   7
Community Score                                      7
Product Responsibility Score                         7
Management Score                                     7
Shareholders Score                                   7
CSR Strategy Score                                   7
Board of Directors/Board Functions (Inactive)        0
Board of D

In [17]:
# Observe ESG data to ensure proper load in data frame, and clean missing information or NAs
ESG = ESG.dropna()
ESG

,Organization ID,Fiscal Year,Special Value Flag,Feed Date,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,...,Asset4 Identifier,Organization ID.1,Refinitiv Business Classification Code,Primary Exchange,Status Flag (1 = Active),Organization Authority Identifier,Ticker,CUSIP,Sedol,ISIN
Fiscal Year End Date,,,,,,,,,,,,,,,,,,,,,
2010-12-25,18227,2010,500,9/25/2022,0.951624,0.515286,0.078947,0.953333,0.992754,0.833333,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2011-12-31,18227,2011,500,9/25/2022,0.905026,0.640013,0.375000,0.980263,0.979452,0.776923,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2012-12-29,18227,2012,500,9/25/2022,0.913084,0.913084,0.958333,0.981250,0.993333,0.778571,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2013-12-28,18227,2013,500,9/25/2022,0.856952,0.653476,0.450000,0.981250,0.993243,0.787671,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2014-12-27,18227,2014,500,9/25/2022,0.913875,0.648114,0.382353,0.981707,0.993421,0.783784,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-29,119895127,2017,500,9/25/2022,0.873850,0.464703,0.055556,0.962500,0.986486,0.424242,...,A4_00069,Cisco Systems Inc,5710,NSM,1,5080018615,CSCO,17275R10,219816,US17275R1023
2018-07-28,119895127,2018,500,9/25/2022,0.882782,0.649724,0.416667,0.989130,0.988636,0.426471,...,A4_00069,Cisco Systems Inc,5710,NSM,1,5080018615,CSCO,17275R10,219816,US17275R1023
2019-07-27,119895127,2019,500,9/25/2022,0.879063,0.879063,0.882353,0.991803,0.973214,0.372093,...,A4_00069,Cisco Systems Inc,5710,NSM,1,5080018615,CSCO,17275R10,219816,US17275R1023


In [ ]:
#Add a risk-free return measure (S&P500) to the Stock Price data


In [ ]:
#Structure the ESG and Stock Price data frames in a similar fashion to facilitate analysis
